# Get Data from Excel

I am going to continue to use excel as the data entry user interface.

In [8]:
import pandas as pd

# Read data from Google Sheets
# url = f"https://docs.google.com/spreadsheets/d/1GuHkRddZm5Idxfa-IUlA-lXQFydduyau/gviz/tq?tqx=out:csv&sheet=Billing"
# df = pd.read_csv(url)

# Read data as excel file
# curl -L https://docs.google.com/spreadsheets/d/1GuHkRddZm5Idxfa-IUlA-lXQFydduyau/export?format=xlsx&sheet=Billing --output billing.xlsx

# read excel file billing.xlsx with the following instructions
# - first lin is blank ignore
# - 2, 3, 4, 5, 6 are are headers
df_excel_billing = pd.read_excel("billing.xlsx", header=[1, 2, 3, 4, 5])

df_excel_billing

Which Unit                         Dates  \
                     Table header                         Dates   
   Which PDF to look at for data?         Bill / Detail of Bill   
            What page to look at?   Generation Charges / Page 1   
                  Table subheader Service End Date [Date]\n(44)   
0                             NaN                    2024-05-07   
1                             NaN                    2024-06-06   
2                             NaN                           NaT   
3                             NaN                           NaT   
4                             NaN                    2024-07-08   
5                             NaN                           NaT   
6                             NaN                           NaT   
7                             NaN                    2024-08-06   
8                             NaN                           NaT   
9                             NaN                           NaT   
10                            NaN                    2024-09-05   
11                            NaN                           NaT   
12                            NaN                           NaT   
13                            NaN                           NaT   
14                            NaN                           NaT   
15                            NaN                    2024-10-06   
16                            NaN                           NaT   
17                            NaN                           NaT   
18                            NaN                    2024-11-04   
19                            NaN                    2024-12-05   
20                            NaN                    2025-01-06   
21                            NaN                           NaT   
22                            NaN                           NaT   
23                            NaN                    2025-02-04   
24                            NaN                    2025-03-06   
25                            NaN                           NaT   
26                            NaN                           NaT   
27                            NaN                    2025-04-06   

                                                              303 Main  \
                             Meter Export Energy (Solar) \n(channel 2)   
                        Bill                          n/a - calculated   
                      Header                        Unnamed: 3_level_3   
   Billing Date [Date]\n(43)                      off peak [kWh]\n(1)    
0        2024-05-14 00:00:00                                    -884.0   
1        2024-06-13 00:00:00                                   -1411.0   
2                     winter                                       NaN   
3                     summer                                       NaN   
4        2024-07-15 00:00:00                                   -1547.0   
5        07 - earlier period                                       NaN   
6          07 - later period                                       NaN   
7        2024-08-16 00:00:00                                   -1214.0   
8            08 - old rates                                        NaN   
9             08 - new rates                                       NaN   
10       2024-09-17 00:00:00                                   -1339.0   
11  summer - old rate period                                       NaN   
12  summer - new rate period                                       NaN   
13                    summer                                       NaN   
14                    winter                                       NaN   
15       2024-10-16 00:00:00                                   -1199.0   
16                    summer                                       NaN   
17                    winter                                       NaN   
18       2024-11-14 00:00:00                                    -973.0   
19       2024-12-17 00:00:00       

# Create structure for storing & analyzing the data

Choosing to do an object oriented class structure because will want to have lots of little nested functions to help me out. 

In [9]:
from dataclasses import dataclass
from typing import List, Optional
from enum import Enum
import datetime
from typing import List
import calendar

class NEM2A_MeterType(Enum):
    GenerationMeter = 1
    BenefitMeter = 2

class WhereFrom(Enum):
    PDF_BILL = 1
    PDF_DETAIL_OF_BILL = 2
    CALCULATED = 3
    FIXED_VALUE = 4
    NOT_PROVIDED = 5


# Stores information about where something is found in the PG&E billing system.
# It could be for many types of information, such as a date, a kWh value, or a cost.
@dataclass
class WhereFound:
    where_from: Optional[WhereFrom] = None    # tells the person where to find this on the bill
    where_on_pdf: Optional[str] = None        # tells the person where to find this on the bill
    kevins_number_code: Optional[int] = None  # a number that Kevin can use to identify this metric by looking at the november 2024 sample bill

# Represents a date and where to find that date in the PG&E billing system.
class EnergyDate:
    value: Optional[datetime.date] = None
    where_found: List[WhereFound] = None

    def __init__(self, value: Optional[datetime.date] = None, where_found: Optional[List[WhereFound]] = None):
        self.value = value
        self.where_found = where_found if where_found is not None else []

# Represents an energy or cost metric with value(s), unit, and where it was found.
class EnergyMetric:

    # Sometimes each metric comes from multiple values summed up.
    # This can happen when the rates change half way through the month (due to summer/winter trasitions or rake hikes)
    subcomponent_values: List[float] = None

    # The unit of the metric, such as kWh or $
    unit: Optional[str] = None

    # Where this metric was found in the PG&E billing system.
    where_found: List[WhereFound] = None

    def __init__(self, unit: Optional[str] = None):
        self.subcomponent_values = []
        self.unit = unit
        self.where_found = []

# Some energy metrics are time-of-use (TOU) based, meaning they have different values for peak and off-peak times.
class EnergyMetricTOU:
    peak: EnergyMetric
    off_peak: EnergyMetric
    total: EnergyMetric

    # constructor which initializes the value to None, unit to None, and where_found to an empty list
    def __init__(self, unit: Optional[str]):
        self.peak = EnergyMetric(unit)
        self.off_peak = EnergyMetric(unit)
        self.total = EnergyMetric(unit)

# Represents a monthly bill for a single meter within the NEM2A system.
class MeterBillingMonth:

    # The type of meter, either GenerationMeter or BenefitMeter
    nem2a_meter_type                : NEM2A_MeterType

    # key dates
    billing_date                    : EnergyDate
    service_end_date                : EnergyDate

    # Meter values (kWh) are always mapped to time of used.
    energy_export_meter_channel_2   : EnergyMetricTOU
    energy_import_meter_channel_1   : EnergyMetricTOU
    allocated_export_credits        : EnergyMetricTOU
    net_energy_usage_after_credits  : EnergyMetricTOU

    # PCE values
    pce_energy_cost                 : EnergyMetricTOU # cost is always TOU based
    pce_net_generation_bonus        : EnergyMetric
    pce_energy_commission_surcharge : EnergyMetric
    pce_total_energy_charges        : EnergyMetric
    pce_nem_credit                  : EnergyMetric
    pce_generation_charges_due_cash : EnergyMetric

    # PG&E
    pge_res_energy_charges          : EnergyMetric
    pge_baseline_credit             : EnergyMetric
    pge_da_cca_charges              : EnergyMetric
    pge_total_energy_charges        : EnergyMetric
    pge_nem_billing                 : EnergyMetric
    pge_minimum_delivery_charge     : EnergyMetric
    pge_nem_true_up_adjustment      : EnergyMetric
    pge_electric_delivery_charges   : EnergyMetric

    # Totals
    california_climate_credit       : EnergyMetric
    total_bill_in_mail              : EnergyMetric

    # Store the meter type as an instance variable
    def __init__(self, nem2a_meter_type: NEM2A_MeterType):

        # Store the meter type
        self.nem2a_meter_type = nem2a_meter_type

        # key dates are the same for both types of meters
        self.billing_date        = EnergyDate(where_found=[WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Header", kevins_number_code=43) ])
        self.service_end_date    = EnergyDate(where_found=[WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges"),
                                                           WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 1", kevins_number_code=44)])

        # Metering units are always kWh
        self.energy_export_meter_channel_2   = EnergyMetricTOU("kWh")
        self.energy_import_meter_channel_1   = EnergyMetricTOU("kWh")
        self.allocated_export_credits        = EnergyMetricTOU("kWh")
        self.net_energy_usage_after_credits  = EnergyMetricTOU("kWh")

        # PCE values always in $
        self.pce_energy_cost                 = EnergyMetricTOU("$")
        self.pce_net_generation_bonus        = EnergyMetric(unit = "$")
        self.pce_energy_commission_surcharge = EnergyMetric(unit = "$")
        self.pce_total_energy_charges        = EnergyMetric(unit = "$")
        self.pce_nem_credit                  = EnergyMetric(unit = "$")
        self.pce_generation_charges_due_cash = EnergyMetric(unit = "$")

        # PG&E values always in $
        self.pge_res_energy_charges          = EnergyMetric(unit = "$")
        self.pge_baseline_credit             = EnergyMetric(unit = "$")
        self.pge_da_cca_charges              = EnergyMetric(unit = "$")
        self.pge_total_energy_charges        = EnergyMetric(unit = "$")
        self.pge_nem_billing                 = EnergyMetric(unit = "$")
        self.pge_minimum_delivery_charge     = EnergyMetric(unit = "$")
        self.pge_nem_true_up_adjustment      = EnergyMetric(unit = "$")
        self.pge_electric_delivery_charges   = EnergyMetric(unit = "$")

        # Totals values always in $
        self.california_climate_credit       = EnergyMetric(unit = "$")
        self.total_bill_in_mail              = EnergyMetric(unit = "$")

        # METER CHANNEL 2: energy exported
        if nem2a_meter_type == NEM2A_MeterType.GenerationMeter:
            self.energy_export_meter_channel_2.off_peak.where_found.append(WhereFound(where_from=WhereFrom.CALCULATED))
            self.energy_export_meter_channel_2.peak.where_found.append(WhereFound(where_from=WhereFrom.CALCULATED))
            self.energy_export_meter_channel_2.total.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 3", kevins_number_code = 15))
        elif nem2a_meter_type == NEM2A_MeterType.BenefitMeter:
            # Never any export on a benefit meter
            self.energy_export_meter_channel_2.off_peak.where_found.append(WhereFound(where_from=WhereFrom.FIXED_VALUE))
            self.energy_export_meter_channel_2.off_peak.subcomponent_values = [0]
            self.energy_export_meter_channel_2.peak.where_found.append(WhereFound(where_from=WhereFrom.FIXED_VALUE))
            self.energy_export_meter_channel_2.peak.subcomponent_values = [0]
            self.energy_export_meter_channel_2.total.where_found.append(WhereFound(where_from=WhereFrom.FIXED_VALUE))
            self.energy_export_meter_channel_2.total.subcomponent_values = [0]

        # METER CHANNEL 1: energy imported
        if nem2a_meter_type == NEM2A_MeterType.GenerationMeter:
            self.energy_import_meter_channel_1.off_peak.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 5", kevins_number_code = 3))
            self.energy_import_meter_channel_1.peak.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 5", kevins_number_code = 4))
            self.energy_import_meter_channel_1.total.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 3", kevins_number_code = 10))
        elif nem2a_meter_type == NEM2A_MeterType.BenefitMeter:
            self.energy_import_meter_channel_1.off_peak.where_found.append(WhereFound(where_from=WhereFrom.CALCULATED))
            self.energy_import_meter_channel_1.peak.where_found.append(WhereFound(where_from=WhereFrom.CALCULATED))
            self.energy_import_meter_channel_1.total.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 3", kevins_number_code = 9))


        # ALLOCATED CREDITS
        if nem2a_meter_type == NEM2A_MeterType.GenerationMeter:
            self.allocated_export_credits.off_peak.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 5", kevins_number_code = 5))
            self.allocated_export_credits.peak.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 5", kevins_number_code = 16))
            self.allocated_export_credits.total.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 3", kevins_number_code = 14))
        elif nem2a_meter_type == NEM2A_MeterType.BenefitMeter:
            self.allocated_export_credits.off_peak.where_found.append(WhereFound(where_from=WhereFrom.CALCULATED))
            self.allocated_export_credits.peak.where_found.append(WhereFound(where_from=WhereFrom.CALCULATED))
            self.allocated_export_credits.total.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 3", kevins_number_code = 13))


        # net energy usage after credits is calculated
        if nem2a_meter_type == NEM2A_MeterType.GenerationMeter:
            # off peak found in 2 places
            self.net_energy_usage_after_credits.off_peak.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges", kevins_number_code = 23))
            self.net_energy_usage_after_credits.off_peak.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 6", kevins_number_code = 23))
            # peak found in 2 places
            self.net_energy_usage_after_credits.peak.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges", kevins_number_code = 22))
            self.net_energy_usage_after_credits.peak.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 6", kevins_number_code = 22))
            # total found in 1 place
            self.net_energy_usage_after_credits.total.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 6", kevins_number_code = 7))
        elif nem2a_meter_type == NEM2A_MeterType.BenefitMeter:
            self.net_energy_usage_after_credits.off_peak.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges", kevins_number_code = 21))
            self.net_energy_usage_after_credits.peak.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges", kevins_number_code = 20))
            self.net_energy_usage_after_credits.total.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 6", kevins_number_code = 8))

        # pce energy cost
        if nem2a_meter_type == NEM2A_MeterType.GenerationMeter:
            self.allocated_export_credits.off_peak.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges", kevins_number_code = 33))
            self.allocated_export_credits.peak.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges", kevins_number_code = 34))
            self.allocated_export_credits.total.where_found.append(WhereFound(where_from=WhereFrom.CALCULATED))
        elif nem2a_meter_type == NEM2A_MeterType.BenefitMeter:
            self.allocated_export_credits.off_peak.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges", kevins_number_code = 24))
            self.allocated_export_credits.peak.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges", kevins_number_code = 25))
            self.allocated_export_credits.total.where_found.append(WhereFound(where_from=WhereFrom.CALCULATED))

        # other pce costs
        if nem2a_meter_type == NEM2A_MeterType.GenerationMeter:
            self.pce_net_generation_bonus.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges", kevins_number_code = 37))
        elif nem2a_meter_type == NEM2A_MeterType.BenefitMeter:
            self.pce_net_generation_bonus.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges", kevins_number_code = 26))

        if nem2a_meter_type == NEM2A_MeterType.GenerationMeter:
            self.pce_energy_commission_surcharge.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges", kevins_number_code = 35))
        elif nem2a_meter_type == NEM2A_MeterType.BenefitMeter:
            self.pce_energy_commission_surcharge.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges", kevins_number_code = 27))

        if nem2a_meter_type == NEM2A_MeterType.GenerationMeter:
            self.pce_total_energy_charges.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges"))
        elif nem2a_meter_type == NEM2A_MeterType.BenefitMeter:
            self.pce_total_energy_charges.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges"))

        if nem2a_meter_type == NEM2A_MeterType.GenerationMeter:
            self.pce_nem_credit.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges"))
        elif nem2a_meter_type == NEM2A_MeterType.BenefitMeter:
            self.pce_nem_credit.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges", kevins_number_code = 28))

        if nem2a_meter_type == NEM2A_MeterType.GenerationMeter:
            self.pce_generation_charges_due_cash.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges"))
        elif nem2a_meter_type == NEM2A_MeterType.BenefitMeter:
            self.pce_generation_charges_due_cash.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Generation Charges", kevins_number_code = 29))

        # pg&e costs that are only found on generation meter side.
        # TODO: figure out how to calcualte these for benefit meter after true up.
        if nem2a_meter_type == NEM2A_MeterType.GenerationMeter:
            self.pge_res_energy_charges.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 4", kevins_number_code = 31))
            self.pge_baseline_credit.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 4", kevins_number_code = 32))
            self.pge_da_cca_charges.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 4", kevins_number_code = 38))
            self.pge_total_energy_charges.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 4", kevins_number_code = 39))
            self.pge_nem_billing.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 4", kevins_number_code = 40))
            self.pge_minimum_delivery_charge.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 4", kevins_number_code = 41))
            self.pge_nem_true_up_adjustment.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 1 - True Up Only", kevins_number_code = 40))
        elif nem2a_meter_type == NEM2A_MeterType.BenefitMeter:
            self.pge_res_energy_charges.where_found.append(WhereFound(where_from=WhereFrom.NOT_PROVIDED))
            self.pge_baseline_credit.where_found.append(WhereFound(where_from=WhereFrom.NOT_PROVIDED))
            self.pge_da_cca_charges.where_found.append(WhereFound(where_from=WhereFrom.NOT_PROVIDED))
            self.pge_total_energy_charges.where_found.append(WhereFound(where_from=WhereFrom.NOT_PROVIDED))
            self.pge_nem_billing.where_found.append(WhereFound(where_from=WhereFrom.NOT_PROVIDED))
            self.pge_minimum_delivery_charge.where_found.append(WhereFound(where_from=WhereFrom.NOT_PROVIDED))
            self.pge_nem_true_up_adjustment.where_found.append(WhereFound(where_from=WhereFrom.NOT_PROVIDED))

        if nem2a_meter_type == NEM2A_MeterType.GenerationMeter:
            self.pge_electric_delivery_charges.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Electric Delivery Charges", kevins_number_code = 30))
            self.pge_electric_delivery_charges.where_found.append(WhereFound(where_from=WhereFrom.PDF_DETAIL_OF_BILL, where_on_pdf="Page 4", kevins_number_code = 30))
        elif nem2a_meter_type == NEM2A_MeterType.BenefitMeter:
            self.pge_electric_delivery_charges.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Electric Delivery Charges", kevins_number_code = 42))

        if nem2a_meter_type == NEM2A_MeterType.GenerationMeter:
            self.california_climate_credit.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Electric Delivery Charges", kevins_number_code = 45))
        elif nem2a_meter_type == NEM2A_MeterType.BenefitMeter:
            self.california_climate_credit.where_found.append(WhereFound(where_from=WhereFrom.PDF_BILL, where_on_pdf="Electric Delivery Charges", kevins_number_code = 46))

        # total bill is calcualted
        self.total_bill_in_mail.where_found.append(WhereFound(where_from=WhereFrom.CALCULATED))

    # function to get the total energy_export_meter_channel_2
    def get_total_energy_export(self):
        """
        Returns the total energy exported in kWh.
        If the meter type is GenerationMeter, it returns the total from channel 2.
        If the meter type is BenefitMeter, it returns 0.
        """

        if self.energy_export_meter_channel_2.total.subcomponent_values:
            return sum(self.energy_export_meter_channel_2.total.subcomponent_values)
        else:
            return 0

    # calculate calculated values for the billing month
    def calculate_values(self):

        # Only calculate for GenerationMeter
        if self.nem2a_meter_type == NEM2A_MeterType.GenerationMeter:
            # For the generator meter the total is "PCE Generation Charges due in Cash (36)" + "PG&E Electric Delivery Charges (30)"  + california_climate_credit (45)
            self.total_bill_in_mail.subcomponent_values.append(
                sum(self.pce_generation_charges_due_cash.subcomponent_values) +
                sum(self.pge_electric_delivery_charges.subcomponent_values) +
                sum(self.california_climate_credit.subcomponent_values)
            )

        # For the benefit meter the total is "PCE Generation Charges due in Cash (29)" + "PG&E Electric Delivery Charges (42)" + california_climate_credit (46)
        elif self.nem2a_meter_type == NEM2A_MeterType.BenefitMeter:
            self.total_bill_in_mail.subcomponent_values.append(
                sum(self.pce_generation_charges_due_cash.subcomponent_values) +
                sum(self.pge_electric_delivery_charges.subcomponent_values) +
                sum(self.california_climate_credit.subcomponent_values)
            )


# Represents a monthly bill for a single meter within the NEM2A system.
class NEM2AAggregationBillingMonth:

    year        : int
    month       : int
    month_label : tuple  # (month_name, year) e.g., ("January", 2024)

    main    : MeterBillingMonth
    adu     : MeterBillingMonth

    def __init__(self, year, month):

        # Store the calendar year and month for this billing month
        # year: e.g., 2024, month: 1=January, 12=December
        if year is not None and month is not None:
            self.year = year
            self.month = month
            self.month_label = (calendar.month_name[month], year)
        else:
            # raise exception if year or month is None
            raise ValueError("Year and month must be provided for MeterBillingMonth initialization.")

        self.main = MeterBillingMonth(nem2a_meter_type=NEM2A_MeterType.GenerationMeter)
        self.adu = MeterBillingMonth(nem2a_meter_type=NEM2A_MeterType.BenefitMeter)

    # function to print the billing month and year on a single line, both as numeric and tuple
    def print_month(self):
        print(f"{self.month_label}")

    # function to return the month as a tuple (year, month numeric)
    def get_month_as_tuple(self):
        return (self.year, self.month)

    # function to get the total energy exported to the grid for this month
    def get_total_energy_export(self):
        return self.main.get_total_energy_export()

    # calculate calculated values for the billing month
    def calculate_values(self):
        self.main.calculate_values()
        self.adu.calculate_values()

# Represents a billing year, which consists of multiple billing months.
class BillingYear:

    start_month : int
    start_year  : int
    num_months  : int
    months      : List[tuple]  # List of tuples (month_name, year)
    billing_months: List[NEM2AAggregationBillingMonth]

    def __init__(self, start_month: int, start_year: int, num_months: int = 12):
        """
        start_month: 1=January, 12=December
        start_year: e.g., 2024
        num_months: number of months in the billing year (default 12)
        """
        self.start_month = start_month
        self.start_year = start_year
        self.num_months = num_months
        self.months = self._generate_months()

        # Create a dictionary where the key is (year, month) and the value is NEM2AAggregationBillingMonth
        self.billing_months: dict = {
            (self.months[i][0], self.months[i][1]): NEM2AAggregationBillingMonth(year=self.months[i][0], month=self.months[i][1])
            for i in range(self.num_months)
        }

    def _generate_months(self):
        months = []
        year = self.start_year
        month = self.start_month
        for _ in range(self.num_months):
            months.append((year, month))
            month += 1
            if month > 12:
                month = 1
                year += 1
        return months

    # function to print out all the billing months in the year
    def print_months(self):
        # self.billing_months is now a dict with keys as (year, month) tuples and values as NEM2AAggregationBillingMonth
        for month_obj in self.billing_months.values():
            month_obj.print_month()

    # function to get all billing months as an array of tuples (year, month numeric)
    def get_months_as_tuples(self):
        # self.billing_months is now a dict with keys as (year, month) tuples and values as NEM2AAggregationBillingMonth
        return list(self.billing_months.keys())

    # function to get the start year of the billing year
    def get_start_year(self):
        return self.start_year

    # get total energy exported to the grid for the entire year
    def get_total_energy_exported(self):
        total_exported = 0
        for month_obj in self.billing_months.values():
            total_exported += month_obj.get_total_energy_export()
        return total_exported

    # calculate all the values for the billing year
    def calculate_values(self):
        for month_obj in self.billing_months.values():
            month_obj.calculate_values()

# Map Excel data into the OOP structure

Using the (#) code system to do this.
This is a work in progress as I'm trying to do this in a way that is not super fragile.

### Support functions to get the path to EnergyMetrics/Dates with a PDF where found location

In [10]:
from typing import Any

# Looks through the energy month object to find all values and their location in the PG&E billing system
# The purpose of getting this into an array is to make it easy to map the excel data into the object structure
def serialize_wherefound(obj: Any, path: str = "", visited=None):
    if visited is None:
        visited = set()
    results = []

    # Avoid infinite recursion by tracking visited objects
    obj_id = id(obj)
    if obj_id in visited:
        return results
    visited.add(obj_id)

    if isinstance(obj, WhereFound):
        # Serialize WhereFound as a dict
        results.append({
            "path": path,
            "where_from": obj.where_from.name if obj.where_from else None,
            "where_on_pdf": obj.where_on_pdf,
            "kevins_number_code": obj.kevins_number_code
        })
    elif isinstance(obj, list):
        for idx, item in enumerate(obj):
            results.extend(serialize_wherefound(item, f"{path}[{idx}]", visited))

    elif hasattr(obj, "__dict__"):
        for attr, value in obj.__dict__.items():
            results.extend(serialize_wherefound(value, f"{path}.{attr}" if path else attr, visited))

    return results

# For the (#) code location in the PG&E PDF bill system mapped to the path of to value in the OOP data structure.
def get_object_path_by_numbercode(object: Any) -> dict:
    """
    Given a NEM2AAggregationBillingMonth object, serialize it and return a dictionary
    mapping kevins_number_code to the path, where found info, and a direct reference
    to the EnergyDate or EnergyMetric object containing the WhereFound.
    """
    def find_object_by_path(obj, path_str):
        """
        Given an object and a dot-separated path (with optional [index]), return the referenced object.
        """
        parts = path_str.split('.')
        current = obj
        for part in parts:
            if '[' in part and ']' in part:
                attr, idx = part[:-1].split('[')
                current = getattr(current, attr)[int(idx)]
            else:
                current = getattr(current, part)
        return current

    # get a list of where founds in the data structure
    serialized = serialize_wherefound(object)

    object_path_by_numbercode = {}

    # Look through the wherefounds
    for item in serialized:

        # Only consider items that have a kevins_number_code and are from PDF_BILL or PDF_DETAIL_OF_BILL
        if item["where_from"] in [WhereFrom.PDF_BILL.name, WhereFrom.PDF_DETAIL_OF_BILL.name]:
            if item["kevins_number_code"] is not None:

                # The path is to the WhereFound object, so get the parent EnergyDate/EnergyMetric
                path_parts = item["path"].split('.')
                # Remove the last part (should be 'where_found[index]')
                parent_path = '.'.join(path_parts[:-1])
                parent_obj = find_object_by_path(object, parent_path)
                # Add the parent_obj reference to the item
                item_with_ref = dict(item)
                item_with_ref["ref_obj"] = parent_obj
                object_path_by_numbercode[item["kevins_number_code"]] = item_with_ref

    return object_path_by_numbercode


# Write a test function to verify the serialization works as expected
def test_serialize_wherefound():

    # Create a sample billing month object
    billingYear = BillingYear(start_month=5, start_year=2024)
    billingYear.billing_months[(2025, 4)].main.energy_export_meter_channel_2.total.subcomponent_values = [100]

    # Serialize it
    object_path_by_numbercode = get_object_path_by_numbercode(billingYear.billing_months[(2025, 4)])

    # Check that the (#) code 10 point to the correct path and has the right value
    assert object_path_by_numbercode[15]["ref_obj"].subcomponent_values == [100]

    # check that it works in the reverse direction
    object_path_by_numbercode[15]["ref_obj"].subcomponent_values = [200]

    # Verify the change is reflected in the original object
    assert billingYear.billing_months[(2025, 4)].main.energy_export_meter_channel_2.total.subcomponent_values == [200]

# Run the test
test_serialize_wherefound()

### Support functions to map the Excel df to the OOP data structure

In [11]:
import pandas as pd
import re
from typing import Any


# Find the (#) codes by Excel dataframe column
def extract_headers_by_number_code(df):
    """
    Given a pandas DataFrame with multi-level columns, extract a dictionary mapping
    (#) number codes to their column tuples.

    Args:
        df: pandas DataFrame with multi-level columns.

    Returns:
        headers_by_number_code: dict mapping number code (int) -> column tuple.
    """
    headers_by_number_code = {}
    for col in df.columns:
        # look through the col for an integer number inside of parenthesis like (#)
        match = re.search(r"\((\d+)\)", str(col))
        if match:
            # Store as a key-value pair: number -> column name (tuple)
            headers_by_number_code[int(match.group(1))] = col
    return headers_by_number_code

# Groups excel rows by month
def group_rows_by_billing_month(excel_dates, billing_year):
    """
    Groups row indices by (year, month) tuple for the given billing year.

    Args:
        excel_dates: pandas Series of dates (can include NaT).
        billing_year: BillingYear object.

    Returns:
        month_groups: dict mapping (year, month) -> list of row indices.
    """
    # Get all months as (year, month) tuples from the BillingYear object
    months_as_tuples = billing_year.get_months_as_tuples()

    # Initialize a dictionary to hold row indices for each month
    month_groups = {month_tuple: [] for month_tuple in months_as_tuples}
    current_month = ()

    # Iterate through the excel_dates Series to assign each row to a billing month
    for i, date in enumerate(excel_dates):

        # If the date is a valid Timestamp or datetime.date and not NaT
        if isinstance(date, (pd.Timestamp, datetime.date)) and not pd.isna(date):
            month_tuple = (date.year, date.month)

            if month_tuple not in month_groups:
                # Optionally log or warn here if the month is not in the billing year
                continue
            current_month = month_tuple
            month_groups[current_month].append(i)

        else:
            # If the date is NaT, assign the row to the most recent month found
            if current_month != ():
                month_groups[current_month].append(i)
    return month_groups

# helper function to make the data frame
def extract_values_by_month_group(rows_assigned_to_each_month, headers_by_number_code, df_excel_billing):
    """
    Given rows_assigned_to_each_month (dict of (year, month) -> list of row indices), headers_by_number_code (dict),
    and the excel dataframe, return a DataFrame with all number codes as columns, indexed by (Month, Excel Row #).
    """

    data = []
    number_codes = list(headers_by_number_code.keys())
    for month in rows_assigned_to_each_month:
        month_rows = rows_assigned_to_each_month[month]
        for row in month_rows:
            row_data = {
                "Month": month,
                "Excel Row #": row
            }
            for code in number_codes:
                col = headers_by_number_code[code]
                value = df_excel_billing.iloc[row][col] if col in df_excel_billing.columns else None
                row_data[code] = value
            data.append(row_data)
    extracted_values_df = pd.DataFrame(data)
    extracted_values_df.set_index(["Month", "Excel Row #"], inplace=True)
    return extracted_values_df

# generates a dataframe based on excel, billing year month and (#) codes
def generate_dataframe_for_billing_year_from_excel(billing_year, df_excel_billing, headers_by_number_code):
    """
    Generates a DataFrame for the given billing year using the provided Excel billing data.
    Args:
        billing_year: BillingYear object.
        df_excel_billing: DataFrame containing the Excel billing data.
        headers_by_number_code: Dictionary mapping number codes to column names.
    Returns:
        DataFrame with all number codes as columns, indexed by (Month, Excel Row #).
    """

    # EXCEL FILE: Get all rows in the Service End dates column
    service_end_dates = df_excel_billing[headers_by_number_code[44]]

    # figure out which rows in the excel file belong to which month
    rows_assigned_to_each_month = group_rows_by_billing_month(service_end_dates, billing_year)

    # Extrack all values for each month and return a DataFrame
    extracted_values_df = extract_values_by_month_group(rows_assigned_to_each_month, headers_by_number_code, df_excel_billing)

    return extracted_values_df


### Copy the Excel data into the OOP data strcture

In [12]:
# EXCEL: Extract headers by number code from the excel billing DataFrame
headers_by_number_code = extract_headers_by_number_code(df_excel_billing)

# OOP STRUCTURE: generate a DataFrame based on the 2024 billing year period using the excel data
billing_year_2024 = BillingYear(start_month=5, start_year=2024)
billing_year_2024_df = generate_dataframe_for_billing_year_from_excel(billing_year_2024, df_excel_billing, headers_by_number_code)

# Print the DataFrame for the billing year 2024
billing_year_2024_df

# Loop through the DataFrame and map values to the OOP structure
count_valid_value_added = 0
count_invalid_values_not_added = 0
previous_month_tuple = ()
for (month_tuple, excel_row), row in billing_year_2024_df.iterrows():

    # Each month will have one object per number code
    object_path_by_numbercode = get_object_path_by_numbercode(billing_year_2024.billing_months[month_tuple])

    # There is only one set of dates per month.
    if month_tuple != previous_month_tuple:
        dates_filled_in = False

    # Loop through the number codes in the row
    for number_code in headers_by_number_code.keys():

        # get the path for the number code
        if number_code not in object_path_by_numbercode:
            continue

        path_info = object_path_by_numbercode[number_code]

        if type(path_info['ref_obj']) == EnergyDate:

            # if it's a valid date store it.
            if pd.notna(row[number_code]) and isinstance(row[number_code], (pd.Timestamp, datetime.date)):
                path_info['ref_obj'].value = pd.to_datetime(row[number_code]).date()
                count_valid_value_added += 1
            elif not dates_filled_in:
                # Throw exception if the value is NaN or not a valid date
                print(f"Processing month: {month_tuple}. Excel Row: {excel_row}: Invalid DATE value for {path_info['path']}: {row[number_code]}")
                count_invalid_values_not_added += 1

        elif type(path_info['ref_obj']) == EnergyMetric:

            # check if the value is not NaN and is a valid number
            if pd.notna(row[number_code]):

                if isinstance(row[number_code], (int, float)):
                    # add the value to the subcomponent values
                    path_info['ref_obj'].subcomponent_values.append(row[number_code])
                    count_valid_value_added += 1

                else:
                    # Throw exception if the value is not valid number
                    print(f"Processing month: {month_tuple}. Excel Row: {excel_row}: Invalid NUMERIC value for {path_info['kevins_number_code']}: {row[number_code]}")
                    count_invalid_values_not_added += 1

    # Once we've gone through all number codes for a month
    dates_filled_in = True
    previous_month_tuple = month_tuple

# calculate all the values for the billing year
billing_year_2024.calculate_values()

print(f"Total valid values added: {count_valid_value_added}")
print(f"Total invalid values not added: {count_invalid_values_not_added}")

Total valid values added: 512
Total invalid values not added: 0


### Some tests to ensure the import worked correctly

In [13]:
# get the total energy exported for the main meter in the billing year
print(f"Value for May 2024 bill channel 1 import total (10) is: {billing_year_2024.billing_months[(2024, 5)].main.energy_import_meter_channel_1.total.subcomponent_values}")

print(f"Value for March 2025 bill channel 1 import peak (4) is: {billing_year_2024.billing_months[(2025, 3)].main.energy_import_meter_channel_1.peak.subcomponent_values}")

print(f"Total exported energy for the year 2024: {billing_year_2024.get_total_energy_exported()} kWh")

Value for May 2024 bill channel 1 import total (10) is: [382.0]
Value for March 2025 bill channel 1 import peak (4) is: [286.0, 204.0, 82.0]
Total exported energy for the year 2024: -12394.0 kWh


# Print out OOP data structure

Print out the OOP data structure in a way that helps me verify the data is correct

Would be good for this to be an HTML table.

In [ ]:
# print out the OOP structure for the billing year 2024 in HTML format
from IPython.display import display, HTML

def print_billing_year_html(billing_year):
    """
    Return the billing year object as HTML table format with main and adu as top-level headings.
    Units are shown beside the numbers, not in the heading.
    Also outputs the "where from" information for each data column.
    """
    # Helper to get "where from" for a metric
    def get_where_from(metric):
        if hasattr(metric, "where_found") and metric.where_found:
            return ", ".join([wf.where_from.name if wf.where_from else "" for wf in metric.where_found])
        return ""

    html_output = f"<h2>Billing Year: {billing_year.get_start_year()}</h2>"
    html_output += """
    <table border="1" style="border-collapse:collapse;">
        <thead>
            <tr>
                <th>month_label</th>
                <th colspan="3" style="text-align:center;">main</th>
                <th colspan="3" style="text-align:center;">adu</th>
            </tr>
            <tr>
                <th></th>
                <th>energy_export_meter_channel_2.total</th>
                <th>energy_import_meter_channel_1.total</th>
                <th>total_bill_in_mail</th>
                <th>energy_export_meter_channel_2.total</th>
                <th>energy_import_meter_channel_1.total</th>
                <th>total_bill_in_mail</th>
            </tr>
            <tr>
                <th></th>
                <th>{main_export_where}</th>
                <th>{main_import_where}</th>
                <th>{main_bill_where}</th>
                <th>{adu_export_where}</th>
                <th>{adu_import_where}</th>
                <th>{adu_bill_where}</th>
            </tr>
        </thead>
        <tbody>
    """.format(
        main_export_where=get_where_from(billing_year.billing_months[next(iter(billing_year.billing_months))].main.energy_export_meter_channel_2.total),
        main_import_where=get_where_from(billing_year.billing_months[next(iter(billing_year.billing_months))].main.energy_import_meter_channel_1.total),
        main_bill_where=get_where_from(billing_year.billing_months[next(iter(billing_year.billing_months))].main.total_bill_in_mail),
        adu_export_where=get_where_from(billing_year.billing_months[next(iter(billing_year.billing_months))].adu.energy_export_meter_channel_2.total),
        adu_import_where=get_where_from(billing_year.billing_months[next(iter(billing_year.billing_months))].adu.energy_import_meter_channel_1.total),
        adu_bill_where=get_where_from(billing_year.billing_months[next(iter(billing_year.billing_months))].adu.total_bill_in_mail),
    )
    for month_tuple in billing_year.get_months_as_tuples():
        month_obj = billing_year.billing_months[month_tuple]
        # Main meter values
        main_export = month_obj.main.get_total_energy_export()
        main_export_unit = month_obj.main.energy_export_meter_channel_2.total.unit or ""
        main_import = sum(month_obj.main.energy_import_meter_channel_1.total.subcomponent_values) if month_obj.main.energy_import_meter_channel_1.total.subcomponent_values else 0
        main_import_unit = month_obj.main.energy_import_meter_channel_1.total.unit or ""
        main_bill = sum(month_obj.main.total_bill_in_mail.subcomponent_values) if month_obj.main.total_bill_in_mail.subcomponent_values else 0
        main_bill_unit = month_obj.main.total_bill_in_mail.unit or ""
        # ADU meter values
        adu_export = month_obj.adu.get_total_energy_export()
        adu_export_unit = month_obj.adu.energy_export_meter_channel_2.total.unit or ""
        adu_import = sum(month_obj.adu.energy_import_meter_channel_1.total.subcomponent_values) if month_obj.adu.energy_import_meter_channel_1.total.subcomponent_values else 0
        adu_import_unit = month_obj.adu.energy_import_meter_channel_1.total.unit or ""
        adu_bill = sum(month_obj.adu.total_bill_in_mail.subcomponent_values) if month_obj.adu.total_bill_in_mail.subcomponent_values else 0
        adu_bill_unit = month_obj.adu.total_bill_in_mail.unit or ""

        html_output += "<tr>"
        html_output += f"<td>{month_obj.month_label[0]} {month_obj.month_label[1]}</td>"
        html_output += f"<td>{main_export} {main_export_unit}</td>"
        html_output += f"<td>{main_import} {main_import_unit}</td>"
        html_output += f"<td>{main_bill} {main_bill_unit}</td>"
        html_output += f"<td>{adu_export} {adu_export_unit}</td>"
        html_output += f"<td>{adu_import} {adu_import_unit}</td>"
        html_output += f"<td>{adu_bill} {adu_bill_unit}</td>"
        html_output += "</tr>"
    html_output += """
        </tbody>
    </table>
    """
    return html_output

html_content = print_billing_year_html(billing_year_2024)

# Display the billing year HTML in the notebook
display(HTML(html_content))

# Save the same HTML to a file in the ../python-app/ folder
with open("../python-app/billing_data_table.html", "w", encoding="utf-8") as f:
    f.write(html_content)


Next things to do:
- Go to Tara & Pirouz house to look at September 2025 to see why the numbers on the Detail of Bill and Main Unit bill don't line up.
- print out information on subcomponent kWh values. Make sure the function is clearly importing the subcomponents and is not double counting the total
- See if we have enough equations to solve for the unknown subcomponent kWh ADU allocated export kWhs. (red cells)